In [ ]:
%reload_ext setup

## Demo

In [ ]:
from igent.workflow import run_workflow

await run_workflow(
    model="openai", # or "azure"
    business_line="enuk",
    registrations_file="../data/demo/registrations_enuk.json",
    offers_file="../data/demo/offers_enuk.json",
    incentives_file="../data/demo/incentives.json",
    matches_file="../data/demo/matches_enuk.json",
    pos_file="../data/demo/pos_enuk.json"
)